# Deck 195 ID Experiment

K.Wood

The ICOADS material you provided for Typhoon Cobra did yield an ID prefix for DEs (Destroyer Escorts) = 14nnn and the cruisers (CA and CL) = 02nnn and 06nnn. I think aircraft carriers (CV and CVE) = 07nnn and 10nnn. Battleships are certainly 01nnn. 

The DCK 195 punch card metadata show potential prefixes are 01-99 (pp-pp), and hull numbers 001-999 (nnn-nnn). 

Now we know the combatant ships are in the small numbers, probably in some kind of grouping, as battleships and cruisers -- aircraft carriers -- destroyers and destroyer escorts, followed by auxiliaries. These latter are mostly "A" designated ships - AK (cargo), AO (fleet oiler) for example, and I suspect the ID numbers are on high end -- say 50-99.  

Would it be possible to extract a) the set of unique prefixes and b) unique prefixes + hull numbers into a table, perhaps with a count of obs associated with the ID and the dates of first and last occurrence? That would be   

a) pp(1-n) 
b) ppnnn, count, date(1), date (n)

I think that would reveal whether there are blocks of IDs associated with different large classes, and the fleet composition in the dataset. On the latter, probably heavy on DDs and DEs. Separately, I have the lists of commissioned ships by type from the Navy -- that's the BB and DD letters, and from that we can understand how complete the punch card project was. 



In [20]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
import datetime

from requests.exceptions import HTTPError


In [4]:
server_url = 'http://akutan.pmel.noaa.gov:8080/erddap'

In [5]:
e = ERDDAP(server=server_url)
# searchterm mooring... subset this later
df = pd.read_csv(e.get_search_url(response='csv', search_for='ICOADS'))

In [15]:
print(df['Dataset ID'].values)

['datasets_IMMA_ICOADS_ncei_1600s' 'datasets_IMMA_ICOADS_ncei_1700s'
 'datasets_IMMA_ICOADS_ncei_1800s' 'datasets_IMMA_ICOADS_ncei_1900s'
 'datasets_IMMA_ICOADS_ncei_1910s' 'datasets_IMMA_ICOADS_ncei_1920s'
 'datasets_IMMA_ICOADS_ncei_1930s' 'datasets_IMMA_ICOADS_ncei_1950s'
 'datasets_IMMA_ICOADS_ncei_1960s' 'datasets_IMMA_ICOADS_ncei_1970s'
 'datasets_IMMA_ICOADS_ncei_1980s' 'datasets_IMMA_ICOADS_ncei_1990s'
 'datasets_IMMA_ICOADS_ncei_2000s' 'datasets_IMMA_ICOADS_ncei_2010s'
 'datasets_IMMA_ICOADS_ncei_1600' 'datasets_IMMA_ICOADS_ncei_1700'
 'datasets_IMMA_ICOADS_ncei_1800' 'datasets_IMMA_ICOADS_ncei_1900'
 'datasets_IMMA_ICOADS_ncei_1910' 'datasets_IMMA_ICOADS_ncei_1920'
 'datasets_IMMA_ICOADS_ncei_1930' 'datasets_IMMA_ICOADS_ncei_1940s'
 'datasets_IMMA_ICOADS_ncei_1950' 'datasets_IMMA_ICOADS_ncei_1960'
 'datasets_IMMA_ICOADS_ncei_1970' 'datasets_IMMA_ICOADS_ncei_1980'
 'datasets_IMMA_ICOADS_ncei_1990' 'datasets_IMMA_ICOADS_ncei_2000'
 'datasets_IMMA_ICOADS_ncei_2010']


In [17]:
kw = {
}

constraints = {
    'DCK>=': "195",
    'DCK<=': "195",    
}


In [188]:
temp_data = ['datasets_IMMA_ICOADS_ncei_1950s']

In [189]:
for dataset_id in temp_data:
        #read and import dataset
        print(dataset_id)
        try:
            d = ERDDAP(server=server_url,
                protocol='tabledap',
                response='csv',
            )
            d.constraints=constraints
            d.dataset_id=dataset_id
        except HTTPError:
            print('Failed to generate url {}'.format(dataset_id))

        try:
            df_m = d.to_pandas(
                        #index_col='time (UTC)',
                        parse_dates=True,
                        skiprows=(1,)  # units information can be dropped.
                        )
            df_m.sort_index(inplace=True)
            df_m.columns = [x[1].split()[0] for x in enumerate(df_m.columns)]
            df_m['ID'] = df_m['ID'].astype(str)

        except:
            print(f"something failed in data download {dataset_id}")
            pass

datasets_IMMA_ICOADS_ncei_1950s
something failed in data download datasets_IMMA_ICOADS_ncei_1950s


In [190]:
#clean ids of decimals and other integer artifacts
df_m['ID_clean']=''
#add unique prefixes and 3dig suffixes
df_m['ID_pre']=''
df_m['ID_suf']=''
for index, row in df_m.iterrows():
    df_m.at[index, 'ID_clean']=row['ID'].split('.')[0].strip().zfill(5)
    df_m.at[index, 'ID_pre']=row['ID'].split('.')[0].strip().zfill(5)[0:2]
    df_m.at[index, 'ID_suf']=row['ID'].split('.')[0].strip().zfill(5)[2:]
    



In [191]:
#unique ship id's
uIDg = df_m.groupby('ID_clean')

In [192]:
print(df_m.groupby('ID_clean').size())

ID_clean
-1292    201
-1318    356
0 010      1
00007      1
00802      1
        ... 
98142    206
98145    666
J1015     53
T4257      1
ZV003      2
Length: 1667, dtype: int64


In [193]:
for groupname in uIDg.groups:
    print(groupname, df_m.groupby('ID_clean').get_group(groupname)['time'].count(), df_m.groupby('ID_clean').get_group(groupname)['time'].min(), df_m.groupby('ID_clean').get_group(groupname)['time'].max())

-1292 201 1945-01-01T14:00:00Z 1945-03-14T17:00:00Z
-1318 356 1944-12-31T21:00:00Z 1945-06-30T10:00:00Z
0 010 1 1945-01-04T09:00:00Z 1945-01-04T09:00:00Z
00007 1 1945-05-01T14:00:00Z 1945-05-01T14:00:00Z
00802 1 1943-04-07T04:00:00Z 1943-04-07T04:00:00Z
00nan 929 1942-01-04T02:00:00Z 1945-12-19T04:00:00Z
01033 791 1942-01-01T12:00:00Z 1945-06-14T11:00:00Z
01034 929 1943-01-01T09:00:00Z 1945-06-30T23:00:00Z
01035 293 1944-01-01T12:00:00Z 1945-05-16T12:00:00Z
01036 889 1943-01-01T16:00:00Z 1945-06-30T11:00:00Z
01038 1075 1943-02-07T16:00:00Z 1945-07-01T04:00:00Z
01040 772 1944-01-01T18:00:00Z 1946-01-01T01:00:00Z
01041 704 1942-01-07T01:00:00Z 1946-01-01T01:00:00Z
01042 639 1943-01-14T04:00:00Z 1945-01-01T04:00:00Z
01044 577 1944-01-01T16:00:00Z 1945-06-30T11:00:00Z
01045 674 1942-01-01T16:00:00Z 1944-06-27T09:00:00Z
01046 659 1944-01-04T16:00:00Z 1945-07-01T04:00:00Z
01056 289 1943-01-02T01:00:00Z 1944-04-01T04:00:00Z
01058 491 1943-12-31T21:00:00Z 1945-12-01T04:00:00Z
01059 243 1944-01

In [180]:
groupname

'-1'